In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data_dir = '/content/drive/MyDrive/digital_twin/Chili_Plant_Disease/train'
valid_data_dir = '/content/drive/MyDrive/digital_twin/Chili_Plant_Disease/val'

In [ ]:
img_width, img_height = 224, 224
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 161 images belonging to 2 classes.


In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 20 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(img_width, img_height, 3)
    ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10
5/5 [==============================] - 46s 6s/step - loss: 0.6279 - accuracy: 0.6899
Epoch 2/10
5/5 [==============================] - 30s 6s/step - loss: 0.2165 - accuracy: 0.8915
Epoch 3/10
5/5 [==============================] - 30s 6s/step - loss: 0.3473 - accuracy: 0.9225
Epoch 4/10
5/5 [==============================] - 29s 7s/step - loss: 0.1426 - accuracy: 0.9380
Epoch 5/10
5/5 [==============================] - 29s 6s/step - loss: 0.1792 - accuracy: 0.9147
Epoch 6/10
5/5 [==============================] - 29s 7s/step - loss: 0.0904 - accuracy: 0.9612
Epoch 7/10
5/5 [==============================] - 30s 5s/step - loss: 0.0637 - accuracy: 0.9845
Epoch 8/10
5/5 [==============================] - 30s 5s/step - loss: 0.0776 - accuracy: 0.9612
Epoch 9/10
5/5 [==============================] - 30s 6s/step - loss: 0.1506 - accuracy: 0.9457
Epoch 10/10
5/5 [==============================] - 38s 7s/step - loss: 0.0945 - accuracy: 0.9563


In [ ]:
model.save('plant_health_model.h5')

testing


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
model = tf.keras.models.load_model('plant_health_model.h5')

In [ ]:
test_image_path = '/content/drive/MyDrive/digital_twin/Chili_Plant_Disease/train/leaf spot22.jpg'

In [ ]:
img_width, img_height = 224, 224

In [ ]:
test_image = load_img(test_image_path, target_size=(img_width, img_height))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

In [ ]:
predictions = model.predict(test_image)

1/1 [==============================] - 0s 66ms/step


In [ ]:
predicted_class = 'Healthy' if predictions[0][0] < 0.5 else 'Unhealthy'

In [ ]:
prediction_probability = predictions[0][0] if predicted_class == 'Unhealthy' else 1 - predictions[0][0]

In [ ]:
print('Predicted Class:', predicted_class)
print('Prediction Probability:', prediction_probability)

Predicted Class: Healthy
Prediction Probability: 0.999999951191878
